# tf Pre-Trained Models
Recommended to use: 
- efficientnet_v2 if dk what to use  

Models to avoid: 
- vgg
    - slow to train
    - not accurate
- mobilenet
    - unless running on mobile phone

## Things to Note
xOut = Dense(num_of_classes)(x)
- e.g. this case we have 102 classes

If accuracy is bad, can add more Dense layers in between Flatten and output layer

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
import matplotlib.pyplot as plt

## Preprocessing
note: some models, such as ResNet, includes a preprocessing function

In [ ]:
# # FYI, to organise images that are not categorised in folders:
# import os, shutil
# import pandas as pd

# data = pd.read_csv('train.csv', index_col='id')

# Path = ''
# for index in data.index:
#   if not os.path.exists(os.path.join(Path, data['target'][index])):
#     os.mkdir(path=os.path.join(Path, data['target'][index]))
#   shutil.copy(os.path.join(Path, data['Image'][index]), os.path.join(Path, data['target'][index]))

In [ ]:
import os, shutil

for i, v in dict.items():
    os.makedirs(v, exist_ok=True)
    shutil.move('train/' +  i, v+'/'+i)

In [5]:
train = tf.keras.preprocessing.image_dataset_from_directory( # latest tensorflow version uses util
    directory='../Datasets/Oxford Flower split/train',
    labels='inferred',
    label_mode='categorical', # labels are one hot encoded
    image_size=(224, 224), # ideally same as original data
    batch_size=32, # must be greater or equal to batch_size in model.fit()
    shuffle=True # for train and val, leave as True
    )

val = tf.keras.preprocessing.image_dataset_from_directory(
    directory='../Datasets/Oxford Flower split/val',
    labels='inferred',
    label_mode='categorical',
    image_size=(224, 224),
    batch_size=32
    )

test = tf.keras.preprocessing.image_dataset_from_directory(
    directory='../Datasets/Oxford Flower split/test',
    labels='inferred',
    label_mode='categorical',
    image_size=(224, 224),
    batch_size=32
    )

Found 5132 files belonging to 102 classes.
Found 1357 files belonging to 102 classes.
Found 1700 files belonging to 102 classes.


In [6]:
train = train.cache().prefetch(buffer_size = tf.data.AUTOTUNE) # default buffer_size = AUTOTUNE
val = val.cache().prefetch(buffer_size = tf.data.AUTOTUNE)
test = test.cache().prefetch(buffer_size = tf.data.AUTOTUNE)

## Build the Model with pre-trained model
### EfficientNet

In [7]:
# with pre-trained model
xIn = Input((224, 224, 3)) # 3rd input is 3 because some models need rgb
net = tf.keras.applications.efficientnet.EfficientNetB0(weights='imagenet', include_top=False)
x = net(xIn)
x = Flatten()(x)
x = Dense(64, activation='swish')(x)
xOut = Dense(102, activation='softmax')(x)

model = Model(xIn, xOut)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['acc'])
model.summary()

callbacks = [
    tf.keras.callbacks.ModelCheckpoint('./best_model', monitor='val_accuracy', save_best_only=True),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)
]
epochs = 20

history = model.fit(train, batch_size=4, validation_data=val, epochs=epochs, callbacks=callbacks)

In [4]:
gpus = tf.config.list_physical_devices()
#tf.config.set_visible_devices(gpus[0], 'GPU')
gpus

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

num_epochs = range(1, epochs + 1)

plt.plot(num_epochs, acc, 'b', label='Training accuracy')
plt.plot(num_epochs, val_acc, 'r', label='Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.figure()

plt.plot(num_epochs, loss, 'b', label='Training loss')
plt.plot(num_epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.show()

### ResNet50

In [ ]:
# with pre-trained model
xIn = Input((224, 224, 3)) # 3rd input is 3 because some models need rgb
x = tf.keras.applications.resnet50.preprocess_input(xIn) # some newer models don't include preprocessing
resnet_model = tf.keras.applications.resnet50.ResNet50(include_top=False, weights='imagenet') # this model is not a layer
x = resnet_model(xIn)
x = Flatten()(x)
x = Dense(64, activation='swish')(x)
xOut = Dense(102, activation='softmax')(x)

model_res = Model(xIn, xOut)

In [ ]:
model_res.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['acc'])
model_res.summary()

callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)
]
epochs = 10

history_res = model_res.fit(train, batch_size=8, validation_data=val, epochs=epochs, callbacks=callbacks)

In [ ]:
acc = history_res.history['acc']
val_acc = history_res.history['val_acc']
loss = history_res.history['loss']
val_loss = history_res.history['val_loss']

num_epochs = range(1, epochs + 1)

plt.plot(num_epochs, acc, 'b', label='Training accuracy')
plt.plot(num_epochs, val_acc, 'r', label='Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.figure()

plt.plot(num_epochs, loss, 'b', label='Training loss')
plt.plot(num_epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.show()

### Using ImageDataGenerator and EfficientNet
aka Data Augmentation  
note: usually only augment training data, don't augment test and validation data

! Don't blindly apply options in ImageDataGenerator
- see how test data differs from training data, and choose based on that
- if data augmentation is too strong, will harm accuracy instead

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
import matplotlib.pyplot as plt

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        brightness_range=(-0.2, 0.2),
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True, 
        vertical_flip=True)

train_generator = train_datagen.flow_from_directory(
        'C:/Users/mandy/OneDrive/2022 IRS/ML/Datasets/Oxford Flower split/train',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

val = tf.keras.preprocessing.image_dataset_from_directory(
    directory='C:/Users/mandy/OneDrive/2022 IRS/ML/Datasets/Oxford Flower split/val',
    labels='inferred',
    label_mode='categorical',
    image_size=(224, 224),
    batch_size=32
    )

In [ ]:
# with pre-trained model
xIn = Input((224, 224, 3)) # 3rd input is 3 because some models need rgb
net = tf.keras.applications.efficientnet.EfficientNetB0(weights='imagenet', include_top=False)
x = net(xIn)
x = Flatten()(x)
x = Dense(64, activation='swish')(x)
xOut = Dense(102, activation='softmax')(x)

model_gen = Model(xIn, xOut)
model_gen.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['acc'])
model_gen.summary()

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)
]

epochs = 10
history_gen = model_gen.fit(train_generator, steps_per_epoch=5132//32 + 1, epochs=epochs, callbacks=callbacks) 
# when using data generator, don't run val data through model.fit

In [ ]:
model_gen.evaluate(val)

In [ ]:
acc = history_gen.history['acc']
val_acc = history_gen.history['val_acc']
loss = history_gen.history['loss']
val_loss = history_gen.history['val_loss']

num_epochs = range(1, 7)

plt.plot(num_epochs, acc, 'b', label='Training accuracy')
plt.plot(num_epochs, val_acc, 'r', label='Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
# plt.figure()

plt.plot(num_epochs, loss, 'b', label='Training loss')
plt.plot(num_epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.show()
plt.savefig('data_generator_oxford_flowers.png')

## Compare models